In [2]:
from image_handling import *

ModuleNotFoundError: No module named 'paddleocr'

# 1. 이미지 자르기

In [ ]:
slice_image_vertically('Untitled.jpg', 'image', 2000)

# 2. OCR 수행

In [ ]:
result=OCR_image('image', 'result_image',2000)

# 3. 문자가 있는 세로 픽셀을 예측하는 모델 설계

## 3.1 이미지의 세로 1픽셀 당 가로줄 평균 RGB값 구하기

In [ ]:
low_pix_mean=RGB_extract('Untitled.jpg')

In [ ]:
data = pd.DataFrame(low_pix_mean, columns=['R','G','B'])

In [ ]:
data.head()

# 3.2 문자가 있는 바운더리 구하기

`바운더리 좌표`는 왼쪽 모서리부터 시계방향으로 나옴

In [ ]:
print(result)

In [ ]:
boundary=[]
for pair in result:
    top= int(min([pair[1][i][1] for i in range(4)]))+3
    bottom=int(max([pair[1][i][1] for i in range(4)]))-3
    boundary.append((top,bottom))

In [ ]:
boundary

## 3.3 target값 매핑

In [ ]:
data['char']=0
for pair in boundary:
    start, end = pair
    data.iloc[start:end+1]['char']=1

In [ ]:
data.iloc[boundary[13][0]-1:boundary[13][1]+2]

# 4. 이미지 존나 많이 크롤링해서 폴더에 넣어놓고 바운더리 미친듯이 학습

In [12]:
total_df = pd.DataFrame(columns=['R', 'G', 'B'])

In [13]:
train_image_list = os.listdir('train_image')
for image in train_image_list:
    # 이미지 폴더안의 이미지 경로
    image_path=os.path.join('train_image', image)
    # 이미지 픽셀 RGB값으로 데이터프레임 생성
    low_pix_mean=RGB_extract(image_path)
    data = pd.DataFrame(low_pix_mean, columns=['R','G','B'])
    # 이미지 자르기
    slice_image_vertically(image_path, 'image', 2000)
    # ocr 수행
    result=OCR_image('image', 'result_image',2000)
    # 글자 영역 추출
    boundary=[]
    for pair in result:
        top= int(min([pair[1][i][1] for i in range(4)]))+3
        bottom=int(max([pair[1][i][1] for i in range(4)]))-3
        boundary.append((top,bottom))
    # 글자 영역에 맞춰서 타겟값 매핑
    data['char']=0
    for pair in boundary:
        start, end = pair
        data.iloc[start:end+1]['char']=1
    total_df=pd.concat([total_df,data], ignore_index=True)
    print(total_df.tail())
    print("총 데이터 수:",len(total_df))

(860, 16988)
[2024/03/30 22:21:44] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\KHU/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\KHU/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e

In [14]:
total_df.to_csv("RGB_data.csv", index=False)

# 5. 기본 머신러닝 모형으로 예측

In [5]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 633742155245386952
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5758779392
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13332440447845105443
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [1]:
import pandas as pd
data=pd.read_csv(r"C:\Users\KHU\Desktop\4학년 1학기\(화,목) 창의적 종합 설계\RGB_data.csv")

In [2]:
X=data.values[:,:-1]
y=data.values[:,-1]
print('X shape :',X.shape)
print('Y shape :',y.shape)

X shape : (205487, 3)
Y shape : (205487,)


In [3]:
from sklearn.model_selection import train_test_split
from collections import Counter

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=1)
print(Counter(y_train)) 
print(Counter(y_test))

Counter({0.0: 128551, 1.0: 35838})
Counter({0.0: 32138, 1.0: 8960})


In [18]:
from sklearn.utils import resample
import numpy as np
# 클래스 0과 클래스 1을 분리
X_train_class_0 = X_train[y_train == 0]
X_train_class_1 = X_train[y_train == 1]

# 클래스 0과 클래스 1의 샘플 수
n_class_0 = len(X_train_class_0)
n_class_1 = len(X_train_class_1)

# 클래스 0의 샘플 수를 클래스 1과 동일하게 맞추기 위해 무작위 샘플링
X_train_resampled_class_0 = resample(X_train_class_0, n_samples=n_class_1, replace=False, random_state=1)

# 언더샘플링된 클래스 0과 클래스 1을 합치기
X_train = np.concatenate([X_train_resampled_class_0, X_train_class_1])
y_train = np.concatenate([np.zeros(n_class_1), np.ones(n_class_1)])

# 언더샘플링 후 클래스 불균형 확인
print("After undersampling:")
print(Counter(y_train))

After undersampling:
Counter({0.0: 35838, 1.0: 35838})


In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

# XGBoost 모델 정의
xgb = XGBClassifier(n_estimators=1000, 
                    learning_rate=0.01, 
                    subsample=0.8, 
                    colsample_bytree=0.8,
                    objective='binary:logistic', 
                    tree_method='gpu_hist')

n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 교차 검증을 통한 예측값 계산 및 분류 보고서 출력
for i, (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
    print(f"Cross-validation {i+1}:")
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[test_index]
    
    # 모델 훈련
    xgb.fit(X_train_fold, y_train_fold)
    
    # 검증 데이터에 대한 예측값 계산
    y_pred_fold = xgb.predict(X_val_fold)
    
    # 분류 보고서 출력
    report = classification_report(y_val_fold, y_pred_fold)
    print("Classification Report:\n", report)
    print("----------------------------------------------------")


Cross-validation 1:
Classification Report:
               precision    recall  f1-score   support

         0.0       0.75      0.65      0.70      7168
         1.0       0.69      0.79      0.74      7168

    accuracy                           0.72     14336
   macro avg       0.72      0.72      0.72     14336
weighted avg       0.72      0.72      0.72     14336

----------------------------------------------------
Cross-validation 2:
Classification Report:
               precision    recall  f1-score   support

         0.0       0.75      0.65      0.69      7168
         1.0       0.69      0.78      0.73      7167

    accuracy                           0.71     14335
   macro avg       0.72      0.71      0.71     14335
weighted avg       0.72      0.71      0.71     14335

----------------------------------------------------
Cross-validation 3:
Classification Report:
               precision    recall  f1-score   support

         0.0       0.76      0.64      0.70      7168

# 6. 순차적 패턴을 고려하여 LSTM까지 설계

In [ ]:
X=data.values[:,:-1]
y=data.values[:,-1]
print('X shape :',X.shape)
print('Y shape :',y.shape)